In [ ]:
##https://github.com/InFoCusp/tf_cnnvis

##https://github.com/vdumoulin/conv_arithmetic

In [ ]:
##including libs

import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
## load and prepare the data base Fashion MNIST

fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
train_images = train_images/ 255.0
test_images = test_images / 255.0

In [ ]:
class_names=[
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(25):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(train_images[i], cmap=plt.cm.binary)
  plt.xlabel(class_names[train_labels[i]])
plt.show()

In [ ]:
## Definir o modelo CNN

model = keras.Sequential([
    keras.layers.Conv2D(8, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(32,activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
## compilar o modelo

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
## treinar o modelo

train_images = train_images[:,:,:,np.newaxis]
mc = ModelCheckpoint('model.h5', monitor='val_accuracy', mode='max', verbose=1,
                     save_best_only=True)
history = model.fit(train_images, train_labels, epochs=10, validation_split=0.2, callbacks=[mc])

In [ ]:
## extraindo historicos de metricas do treinamento

train_loss = history.history['loss']#perda do trinamento
train_acc = history.history['accuracy']#acuracia de treinamento
valid_loss = history.history['val_loss']#perda de validação
valid_acc = history.history['val_accuracy']#acuracia da validação

In [ ]:
# função para plotar as curvas de aprendizado

def save_plots(train_acc, valid_acc, train_loss, valid_loss):
  plt.figure(figsize=(12,9))
  plt.plot(train_acc, color='green', linestyle='-', label='train accuracy')
  plt.plot(valid_acc, color='blue', linestyle='-', label='validation accuracy')
  plt.xlabel('epochs')
  plt.ylabel('accuracy')
  plt.legend()
  plt.savefig('accuracy.png')
  plt.show()

  plt.figure(figsize=(12,9))
  plt.plot(train_loss, color='orange', linestyle='-', label='train loss')
  plt.plot(valid_loss, color='red', linestyle='-', label='validation loss')
  plt.xlabel('epochs')
  plt.ylabel('loss')
  plt.legend()
  plt.savefig('loss.png')
  plt.show()

save_plots(train_acc, valid_acc, train_loss, valid_loss)

In [ ]:
# avaliar o modelo com os dados de teste

model = load_model('model.h5')
test_images = test_images[:,:,:, np.newaxis]
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"acuracia nos dados de teste: {test_accuracy:.2f}")